In [1]:
using Polynomials
push!(LOAD_PATH, pwd())
using Controlz
using Test
using Gadfly
using DifferentialEquations
using Logging
using DataFrames

hipster_theme = Theme(default_color="orange", line_width=1mm, 
    major_label_font_size=14pt, minor_label_font_size=12pt, 
    panel_fill="ivory", minor_label_font="Courier", 
    major_label_font="Courier", point_size=5pt);

┌ Info: Precompiling Controlz [top-level]
└ @ Base loading.jl:1273
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/cokes/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


In [ ]:
##
# second order, underdamped
K = 4.3
τ = 2.8
ξ = 0.2
g = K / (τ ^ 2 * s ^ 2 + 2 * τ * ξ * s + 1)
# impulse response
t, y = simulate(g, (0.0, 25.0))

plot(x=t, y=y, Geom.line, 
    Guide.xlabel("time, t"), 
    Guide.ylabel("output y(t)"),
    Guide.title("underdamped step response"),
    hipster_theme)

In [ ]:
g = zeros_poles_gain([2*im, -2*im, 3], [4, 5, 6], 13.0)

In [ ]:
z, p, k = zeros_poles_gain(g)

plot(layer(x=real(z), y=imag(z), Geom.point, shape=[Shape.cross]),
     layer(x=real(p), y=imag(p), Geom.point),
     Guide.xlabel("Re"),
     Guide.ylabel("Im"),
        hipster_theme
        )

In [ ]:
real(z)

In [ ]:
plot(layer(x=rand(4), y=rand(4), Geom.point),
     layer(x=rand(4), y=rand(4), Geom.point),
     Guide.xlabel("Re"),
     Guide.ylabel("Im"),
     shape=[Shape.hexagon, Shape.cross]
)

In [ ]:
df = DataFrame(x=rand(3), y=rand(3), z=["a", "a", "b"])

plot(df, x=:x, y=:y, shape=:z, hipster_theme, shapes=[Shape.cross, Shape.circle])